In [25]:
import string

In [26]:
def verifica_digito(character : str):
    digitos = ['0','1','2','3','4','5','6','7','8','9']
    if character in digitos:
        return True
    else:
        return False

In [27]:
def verifica_se_letra(character : str):
    
    if (character in string.ascii_lowercase) or (character in string.ascii_uppercase):
        return True
    else:
        return False

In [28]:
def verifica_existencia_caractere(character : str):
    if (character != None) and not(character.isspace()):
        return True
    else:
        return False

In [29]:
palavras_reservadas = ['inicio', 'varinicio', 'varfim', 'escreva', 'leia', 
                      'se', 'entao', 'fimse', 'fim', 'inteiro', 'literal', 'real']

In [30]:
indice_tabela = {'D':0, '\"':1, 'L':2, '{':3}

In [31]:
indice_tabela['"']

1

In [32]:
def realiza_mudanca_de_estado(estado_atual, character : str):
    character_index = character

    if verifica_digito(character):
        character_index = 'D'

    if verifica_se_letra(character):
        character_index  = 'L'
        
    return tabela_de_estados[estado_atual][indice_tabela[character_index]]

In [33]:
class Token():
    def __init__(self, classe : str, lexema : str, tipo : str) -> None:
        self.classe = classe
        self.lexema = lexema
        self.tipo = tipo
    
    def __str__(self) -> str:
        return f"Classe: {self.classe}, Lexema: {self.lexema}, Tipo: {self.tipo}"
    def __repr__(self) -> str:
        return f"Classe: {self.classe}, Lexema: {self.lexema}, Tipo: {self.tipo}"

In [34]:
x = Token(classe = "Qualquer merda", lexema = "Nem sei o que é", tipo = "2")

In [21]:

from tabela_de_estados import TabelaDeEstados
from Tabela_de_Simbolos import Tabela_de_Simbolos
from Token import Token

class Scanner:
  def __init__(self, codigoFonte):
    self.numero_da_coluna = 0
    self.numero_da_linha = 0
    self.codigoFonte = codigoFonte
    self.tamanhoDoCodigo = len(codigoFonte)
    self.tamanhoDaLinha = len(codigoFonte[self.numero_da_linha])
    self.lista_de_textos = []
    self.lexema = ''
    self.Tabela_de_Simbolos = Tabela_de_Simbolos()


  def scanner(self, tabelaEstados: TabelaDeEstados, arrayDeCaracteres: list[str]):
    print('CODIGO: ', arrayDeCaracteres)
    while(arrayDeCaracteres):
      char = arrayDeCaracteres[0]
      entrada = tabelaEstados.verificaTipoCaractere(char).strip()
      if(entrada == '$'):
        if(tabelaEstados.verificarSeEstaEmEstadoFinal()):
          # Adiciona penúltimo token na tabela de símbolos
          #self.Tabela_de_Simbolos.inserir_token(self.Tabela_de_Simbolos, 
          #                            token=Token(classe = tabelaEstados.retornaClasse(), 
          #                                        lexema = self.lexema, 
          #                                        tipo = tabelaEstados.retornaTipo()))
          print('Classe: ', tabelaEstados.retornaClasse())
          # print('Lexema: ', self.lexema)
          print('Tipo: ', tabelaEstados.retornaTipo())
          print('='*15)
          #self.Tabela_de_Simbolos.inserir_token(self.Tabela_de_Simbolos, token=Token(classe='EOF', lexema='EOF',tipo='NULO'))
          print('Fim de arquivo - Token EOF ')
          arrayDeCaracteres = []
          tabelaEstados.estado_atual = 0
          self.lexema = ''
          return 'FINAL DE ARQUIVO' # Adiciona último token na tabela de símbolos Token(EOF)
        else:
          print('LEXEMA 1: ', lexema)
          tabelaEstados.lancarErro(self.numero_da_linha)
          arrayDeCaracteres.remove(char)
          self.lexema = ''
      elif (tabelaEstados.verificarSeEntradaPertenceAoAlfabeto(entrada)):
        if(tabelaEstados.verificarSeProximoEstadoEValido(entrada)):
          self.lexema = self.lexema + char.strip()
          arrayDeCaracteres.remove(char)
          self.scanner(tabelaEstados, arrayDeCaracteres)
        elif (tabelaEstados.verificarSeEstaEmEstadoFinal()):
          #self.Tabela_de_Simbolos.inserir_token(self.Tabela_de_Simbolos, 
          #                                      token=Token(classe = tabelaEstados.retornaClasse(), 
          #                                                  lexema = self.lexema, 
          #                                                  tipo = tabelaEstados.retornaTipo()))
          print('Classe: ', tabelaEstados.retornaClasse())
          print('Lexema: ', self.lexema)
          print('Tipo: ', tabelaEstados.retornaTipo())
          print('='*15)
          tabelaEstados.estado_atual = 0
          self.lexema = ''
          return entrada # Adiciona token na tabela de símbolos
        else:
          print('LEXEMA 2: ', self.lexema)
          tabelaEstados.lancarErro(self.numero_da_linha)
          arrayDeCaracteres.remove(char)
          self.lexema = ''
      elif(tabelaEstados.entradaVazia(char)):
        arrayDeCaracteres.remove(char)
        print('ENTRADA VAZIA')
        continue
      else:
        self.numero_da_coluna = arrayDeCaracteres.index(char)
        
        print('LEXEMA 3: ', self.lexema)
        print('ERRO LÉXICO – Caracter inválido, linha', self.numero_da_linha)
        print('ERRO LÉXICO – Caracter inválido, COLUNA', self.numero_da_coluna)
        self.lexema = ''
        arrayDeCaracteres.remove(char)
        tabelaEstados.estado_atual = 0


  def limpa_codigo(self, codigoFonteFormatado):
    codigo_final = []
    for line in codigoFonteFormatado:
        for caractere in line:
            codigo_final.append(caractere)
    return codigo_final

  def scannerMain(self, tabelaEstados: TabelaDeEstados, codigoFonte):
    for linha in codigoFonte:
      codigoFormatado = self.limpa_codigo(linha)
      print('-'*30)
      self.numero_da_linha += 1
      print('Linha: ', linha)
      print('NUMERO LINHA: ', self.numero_da_linha)
      self.scanner(tabelaEstados, codigoFormatado)


In [13]:
arquivo = open("teste.txt", "r")
codigoFonte = arquivo.readlines()

def limpa_codigo(codigoFonte):
  novo_codigo = []
  for i in codigoFonte:
    novo_codigo.append(i[:][:-1])
  return novo_codigo
  
codigoFormatado = limpa_codigo(codigoFonte)

codigo_final = []
for line in codigoFormatado:
    for caractere in line:
        codigo_final.append(caractere)

codigoFormatado

['inicio',
 '    varinicio',
 '        literal A;',
 '        inteiro B, D, E;',
 '        real C ;',
 '    varfim;',
 '    escreva "Digite B:";',
 '    leia B;',
 '    escreva "Digite A:";',
 '    leia A;',
 '    se(B>2)',
 '    entao',
 '        se(B<=4)',
 '        entao',
 '            escreva "B esta entre 2 e 4";',
 '        fimse',
 '    fimse',
 '    B<-B+1;',
 '    B<-B+2;',
 '    B<-B+3;',
 '    D<-B;',
 '    C<-5.0;',
 '    E<-B+2;',
 '    escreva C;',
 '    B<-B+1;',
 '    escreva "\\nB=\\n"; {\\n Ã© o sÃ\xadmbolo para salto de linha}',
 '    escreva D;',
 '    escreva "\\n";',
 '    escreva C;',
 '    escreva "\\n";',
 '    escreva A;',
 'fi']

In [22]:
tabelaDeEstados = TabelaDeEstados()
# tabelaDeEstados.entradaVazia('     s     ')
teste = Scanner(codigoFormatado)

# array = [' ', ' ', 'a']
# teste.limpa_codigo(teste.codigoFontePalavra, array)
teste.scannerMain(tabelaDeEstados, codigoFormatado)

------------------------------
Linha:  inicio
NUMERO LINHA:  1
CODIGO:  ['i', 'n', 'i', 'c', 'i', 'o']
CODIGO:  ['n', 'i', 'c', 'i', 'o']
CODIGO:  ['i', 'c', 'i', 'o']
CODIGO:  ['c', 'i', 'o']
CODIGO:  ['i', 'o']
CODIGO:  ['o']
CODIGO:  []
------------------------------
Linha:      varinicio
NUMERO LINHA:  2
CODIGO:  [' ', ' ', ' ', ' ', 'v', 'a', 'r', 'i', 'n', 'i', 'c', 'i', 'o']
ENTRADA VAZIA
ENTRADA VAZIA
ENTRADA VAZIA
ENTRADA VAZIA
CODIGO:  ['a', 'r', 'i', 'n', 'i', 'c', 'i', 'o']
CODIGO:  ['r', 'i', 'n', 'i', 'c', 'i', 'o']
CODIGO:  ['i', 'n', 'i', 'c', 'i', 'o']
CODIGO:  ['n', 'i', 'c', 'i', 'o']
CODIGO:  ['i', 'c', 'i', 'o']
CODIGO:  ['c', 'i', 'o']
CODIGO:  ['i', 'o']
CODIGO:  ['o']
CODIGO:  []
------------------------------
Linha:          literal A;
NUMERO LINHA:  3
CODIGO:  [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'l', 'i', 't', 'e', 'r', 'a', 'l', ' ', 'A', ';']
ENTRADA VAZIA
ENTRADA VAZIA
ENTRADA VAZIA
ENTRADA VAZIA
ENTRADA VAZIA
ENTRADA VAZIA
ENTRADA VAZIA
ENTRADA VAZIA
